# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Shahaf  ->  S. Shahaf  |  ['S. Shahaf']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
C. Doughty  ->  C. Doughty  |  ['C. Doughty']
T. Henning  ->  T. Henning  |  ['T. Henning']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
J. Li  ->  J. Li  |  ['J. Li']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
W. Brandner  ->  W. Brandner  |  ['W. Brandner']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
S. Joharle  ->  S. Joharle  |  ['S. Joharle']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']


J. Sauter  ->  J. Sauter  |  ['J. Sauter']
S. Jiao  ->  S. Jiao  |  ['S. Jiao']
J. Li  ->  J. Li  |  ['J. Li']
N. Wang  ->  N. Wang  |  ['N. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
A. Huber  ->  A. Huber  |  ['A. Huber']
J. Schneider  ->  J. Schneider  |  ['J. Schneider']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
C. Gapp  ->  C. Gapp  |  ['C. Gapp']
Arxiv has 62 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.13413


extracting tarball to tmp_2509.13413...

 done.


S. Shahaf  ->  S. Shahaf  |  ['S. Shahaf']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


...

extracting tarball to tmp_2509.14224...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2509.14224/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'commanddef.tex' from 'tmp_2509.14224/commanddef.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
C. Gapp  ->  C. Gapp  |  ['C. Gapp']


Found 150 bibliographic references in tmp_2509.14224/main.bbl.
Error retrieving bib data for krissansen2024erosion: 'author'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.13413-b31b1b.svg)](https://arxiv.org/abs/2509.13413) | **Gaia Barium Dwarfs and Their Ostensibly Ordinary Counterparts**  |
|| P. Rekhi, et al. -- incl., <mark>S. Shahaf</mark>, <mark>J. Müller-Horn</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-09-18*|
|*Comments*| **|
|**Abstract**|            The recently identified Gaia population of main-sequence--white dwarf (MS+WD) binaries at separations of ${\sim}\,1~{\rm AU}$, often with moderate eccentricities, is not readily reproduced by binary population synthesis models. Barium stars represent a closely related population whose enrichment in $s$-process elements both confirms the presence of a WD companion and attests to past binary interaction. It also indicates that mass transfer occurred at least during the late- and post-AGB phases of the WD progenitor, when $s$-process elements are dredged up. In this work, we further explore the connection between the astrometrically identified Gaia MS+WD binaries and the classical barium star population. To achieve this, we used high-resolution FEROS spectroscopy to measure abundances for 30 Gaia DR3 non-single-star binaries, identifying 10 as Ba-enriched. Together with our recent analysis of archival GALAH data, this yields a sample of 38 barium dwarfs with dynamically measured WD masses, compared to only 6 previously known systems with known WD masses at these separations. We find that, in cases where metallicity is sufficiently low to facilitate efficient $s$-process production, barium and yttrium enrichment is often detected. This enrichment is also identified in eccentric systems, suggesting that post-AGB mass transfer mechanisms are capable of pumping eccentricity into the orbit or occur without erasing it. Our results indicate that the Gaia MS+WD binaries trace the population from which barium stars emerge. Treating the large Gaia-discovered population as an extension of known $s$-process enriched dwarfs opens an avenue to empirically constrain their formation and evolution.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.13587-b31b1b.svg)](https://arxiv.org/abs/2509.13587) | **Direct Imaging Explorations for Companions from the Subaru/IRD Strategic Program II; Discovery of a Brown-dwarf Companion around a Nearby mid-M~dwarf LSPM~J1446+4633**  |
|| T. Uyama, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-09-18*|
|*Comments*| *11 pages, 7 figs, 4 tables, accepted for publication in AJ*|
|**Abstract**|            We report the discovery of a new directly-imaged brown dwarf companion with Keck/NIRC2+pyWFS around a nearby mid-type M~dwarf LSPM~J1446+4633 (hereafter J1446). The $L'$-band contrast ($4.5\times10^{-3}$) is consistent with a $\sim20-60\ M_{\rm Jup}$ object at 1--10~Gyr and our two-epoch NIRC2 data suggest a $\sim30\%$ ($\sim3.1\sigma)$ variability in its $L'$-band flux. We incorporated Gaia DR3 non-single-star catalog into the orbital fitting by combining the Subaru/IRD RV monitoring results, NIRC2 direct imaging results, and Gaia proper motion acceleration. As a result, we derive ${59.8}_{-1.4}^{+1.5}\ M_{\rm Jup}$ and $\approx4.3~{\rm au}$ for the dynamical mass and the semi-major axis of the companion J1446B, respectively. J1446B is one of the intriguing late-T~dwarfs showing variability at $L'$-band for future atmospheric studies with the constrained dynamical mass. Because the J1446 system is accessible with various observation techniques such as astrometry, direct imaging, and high-resolution spectroscopy including radial velocity measurement, it has a potential as a great benchmark system to improve our understanding for cool dwarfs.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.13845-b31b1b.svg)](https://arxiv.org/abs/2509.13845) | **Polycyclic aromatic hydrocarbons destruction in star-forming regions across 42 nearby galaxies**  |
|| O. V. Egorov, et al. -- incl., <mark>K. Kreckel</mark>, <mark>J. Li</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-09-18*|
|*Comments*| *19 pages, 15 figures. Accepted for publication in A&A*|
|**Abstract**|            Polycyclic aromatic hydrocarbons (PAHs) are widespread in the interstellar medium (ISM) of Solar metallicity galaxies, where they play a critical role in ISM heating, cooling, and reprocessing stellar radiation. The PAH fraction, the abundance of PAHs relative to total dust mass, is a key parameter in ISM physics. Using JWST and MUSE observations of 42 galaxies from the PHANGS survey, we analyze the PAH fraction in over 17 000 H II regions spanning a gas-phase oxygen abundance of 12+log(O/H) = 8.0-8.8 (Z ~ 0.2-1.3 Zsun), and ~400 isolated supernova remnants (SNRs). We find a significantly lower PAH fraction toward H II regions compared to a reference sample of diffuse ISM areas at matched metallicity. At 12+log(O/H) > 8.2, the PAH fraction toward H II regions is strongly anti-correlated with the local ionization parameter, suggesting that PAH destruction is correlated with ionized gas and/or hydrogen-ionizing UV radiation. At lower metallicities, the PAH fraction declines steeply in both H II regions and the diffuse ISM, likely reflecting less efficient PAH formation in metal-poor environments. Carefully isolating dust emission from the vicinity of optically-identified supernova remnants, we see evidence for selective PAH destruction from measurements of lower PAH fractions, which is, however, indistinguishable at ~50 pc scales. Overall, our results point to ionizing radiation as the dominant agent of PAH destruction within H II regions, with metallicity playing a key role in their global abundance in galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.13911-b31b1b.svg)](https://arxiv.org/abs/2509.13911) | **Spatially resolved broad line region in a quasar at z=4: Dynamical black hole mass and prominent outflow**  |
|| G. Collaboration, et al. -- incl., <mark>W. Brandner</mark>, <mark>P. Garcia</mark>, <mark>S. Joharle</mark>, <mark>L. Kreidberg</mark>, <mark>J. Sauter</mark> |
|*Appeared on*| *2025-09-18*|
|*Comments*| *submitted to A&A (15 pages, 10 figures)*|
|**Abstract**|            We present the first near-infrared interferometric data of a QSO at z=4. The K-band observations were performed with GRAVITY+ on the VLTI using all 4 UTs, detecting a differential phase signal that traces the spatially resolved kinematics for both the H$\beta$ and H$\gamma$ lines in the broad line region. We fit the two lines simultaneously with an updated model that includes distinct rotating and conical outflowing components. We find that more than 80\% of the HI line emission from the BLR originates in an outflow with a velocity up to $10^4$ km s$^{-1}$. This is oriented so that our line of sight is along an edge of the conical structure, which produces the prominent blue wing on the line profile. A combination of anisotropic line emission and mid-plane opacity lead to the single-sided phase signal. The model is able to qualitatively match both the outflowing CIV line profile and the systemic OI fluorescent emission. The derived black hole mass of $8\times10^8$ M$_\odot$ is the highest redshift black hole mass measurement to date obtained directly from BLR dynamics. It is an order of magnitude lower than that inferred from various single epoch scaling relations, and implies that the accretion is highly super-Eddington. With reference to recent simulations, the data suggest that this QSO is emitting close to its radiative limit in a regime where strong outflows are expected around a polar conical region.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.13498-b31b1b.svg)](https://arxiv.org/abs/2509.13498) | **Using Neural Emulators and Hamiltonian Monte Carlo to constrain the Epoch of Reionization's History with the Ly$α$ Forest Power Spectrum**  |
|| D. González-Hernández, et al. -- incl., <mark>C. Doughty</mark> |
|*Appeared on*| *2025-09-18*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**|            The Lyman-alpha (Ly$\alpha$) forest at $z \sim 5$ offers a primary probe to constrain the history of the Epoch of Reionization (EoR), retaining thermal and ionization signatures imprinted by the reionization process. In this work, we present a new inference framework based on JAX that combines forward-modeled Ly$\alpha$ forest observables with differentiable neural emulators and Hamiltonian Monte Carlo (HMC). We construct a dataset of 501 low-resolution simulations generated with user-defined reionization histories and compute a set of 1D Ly$\alpha$ power spectra and model-dependent covariance matrices. We then train two independent neural emulators that achieve sub-percent errors across relevant scales and combine them with HMC to efficiently perform parameter estimation. We validate this framework by applying it to a suite of mock observations, demonstrating that the true parameters are reliably recovered. While this work is limited by the low resolution of the simulations used, our results highlight the potential of this method for inferring the reionization history from high-redshift Ly$\alpha$ forest measurements. Future improvements in our reionization models will further enhance its ability to extract constraints from observational datasets.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.13812-b31b1b.svg)](https://arxiv.org/abs/2509.13812) | **Fermi-LAT Detected Gamma-ray Emission Likely Associated with SNR Kes 78**  |
|| Y.-Z. Shen, Y. Chen, <mark>X. Zhang</mark>, C. Huang |
|*Appeared on*| *2025-09-18*|
|*Comments*| *To appear in the Astrophysical Journal. 12 pages, 5 figures*|
|**Abstract**|            We have analyzed the GeV gamma-ray emission in the region of the supernova remnant (SNR) Kes 78 using $\sim$16.7 years of Fermi-LAT observations and found that the catalog sources 4FGL J1852.4+0037e and 4FGL J1851.8$-$0007c are better represented as two extended sources modeled as `2Ext'. One of them, designated as E2, is located at R.A.$=282.86^\circ$, Dec.$=-0.11^\circ$ with the 68\% containment radius $R_{68} = 0.31^\circ$, and is detected with a significance of 15.2$\sigma$ in the 0.2--500 GeV energy range. The gamma-ray emission of source E2 is well described by a log-parabola (LogP) spectral model with spectral index $\Gamma$ = 1.2 and curvature $\beta$ = 0.3. The fitting with electron-proton number ratio $K_{\rm ep}=0.01$ indicates that the GeV emission of source E2 is dominated by hadronic emission. Given the dense molecular environment surrounding the middle-aged SNR Kes 78, the hadronic scenario provides a natural explanation for the observed GeV emission. The extended source E2 can also be replaced with two point sources. One of them, designated as PTS1, is coincident with the newly discovered PSR J1852$-$0002g within the 68\% positional uncertainty circle, indicating a possible gamma-ray contribution from this PSR. The gamma-ray spectrum of source PTS1 can be well described by a LogP spectral shape. The synchro-curvature radiation model provides a satisfactory spectral fit for source PTS1, suggesting that some of the GeV emission from the Kes 78 region might possibly originate from the magnetosphere of PSR J1852$-$0002g.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.13983-b31b1b.svg)](https://arxiv.org/abs/2509.13983) | **Scientific Objectives of the Xue-shan-mu-chang 15-meter Submillimeter Telescope**  |
|| X. P. C. Group, et al. -- incl., <mark>S. Jiao</mark>, <mark>J. Li</mark>, <mark>N. Wang</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-09-18*|
|*Comments*| *Accepted by Science China Physics, Mechanics & Astronomy*|
|**Abstract**|            Submillimeter astronomy is poised to revolutionize our understanding of the Universe by revealing cosmic phenomena hidden from optical and near-infrared observations, particularly those associated with interstellar dust, molecular gas, and star formation. The Xue-shan-mu-chang 15-meter submillimeter telescope (XSMT-15m), to be constructed at a premier high-altitude site (4813 m) in Qinghai, China, marks a major milestone for Chinese astronomy, establishing the China mainland's first independently developed, world-class submillimeter facility. Equipped with state-of-the-art instruments, XSMT-15m will address a diverse range of frontier scientific questions spanning extragalactic astronomy, Galactic structure, time-domain astrophysics, and astrochemistry. In synergy with current and forthcoming observatories, XSMT-15m will illuminate the formation and evolution of galaxies, unravel the physical and chemical processes shaping the interstellar medium, and explore transient phenomena in the submillimeter regime. These capabilities will advance our understanding across extragalactic astronomy, Galactic ecology, astrochemistry, and time-domain astrophysics, inaugurating a new era for submillimeter research in China and the northern hemisphere.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.14016-b31b1b.svg)](https://arxiv.org/abs/2509.14016) | **Improving cosmological reach of a gravitational wave observatory using Deep Loop Shaping**  |
|| J. Buchli, et al. -- incl., <mark>A. Huber</mark> |
|*Appeared on*| *2025-09-18*|
|*Comments*| **|
|**Abstract**|            Improved low-frequency sensitivity of gravitational wave observatories would unlock study of intermediate-mass black hole mergers, binary black hole eccentricity, and provide early warnings for multi-messenger observations of binary neutron star mergers. Today's mirror stabilization control injects harmful noise, constituting a major obstacle to sensitivity improvements. We eliminated this noise through Deep Loop Shaping, a reinforcement learning method using frequency domain rewards. We proved our methodology on the LIGO Livingston Observatory (LLO). Our controller reduced control noise in the 10--30Hz band by over 30x, and up to 100x in sub-bands surpassing the design goal motivated by the quantum limit. These results highlight the potential of Deep Loop Shaping to improve current and future GW observatories, and more broadly instrumentation and control systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.14019-b31b1b.svg)](https://arxiv.org/abs/2509.14019) | **The Galaxy Activity, Torus, and Outflow Survey (GATOS): TBD. Unveiling physical processes in local active galaxies. Unsupervised hierarchical clustering of JWST MIRI/MRS observations**  |
|| L. H. Muñoz, et al. -- incl., <mark>J. Schneider</mark> |
|*Appeared on*| *2025-09-18*|
|*Comments*| *Submitted to A&A*|
|**Abstract**|            With the rise of the integral field spectroscopy, we are currently dealing with large amounts of spatially resolved data, whose analysis has become challenging, especially when observing complex objects such as nearby galaxies. We aim to develop a method to automatically separate different physical regions within the central parts (1"~160 pc, on average) of galaxies. This can allow us to better understand the systems, and provide an initial characterisation of the main ionisation sources affecting its evolution. We have developed an unsupervised hierarchical clustering algorithm to analyse data cubes based on spectral similarity. It clusters together spaxels with similar spectra, which is useful to disentangle between different physical processes. We have applied this method to a sample of 15 nearby (distances <100 Mpc) galaxies, 7 from the Galaxy Activity, Torus, and Outflow Survey (GATOS) and 8 archival sources, all observed with the medium resolution spectrometer (MRS) of the Mid-Infrared Instrument (MIRI) on board of the JWST. From the clusters, we computed their median spectrum and measured the line and continuum properties. We used these measurements to train random forest models and create several empirical mid-IR diagnostic diagrams for the MRS channel 3 wavelength range, including among others the bright [Ne II], [Ne III], and [Ne V] lines, several H2 transitions, and PAH features. The clustering technique allows to differentiate emission coming from an AGN, the disc, and star forming regions in galaxies, and other composite regions, potentially ionised by several sources simultaneously. This is supported by the results from the empirical diagnostic diagrams, that are indeed able to separate physically distinct regions. This innovative method serves as a tool to identify regions of interest in any data cube prior to an in-depth analysis of the sources. [abridged]         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.13413/figures/P_e_general.jpg', 'tmp_2509.13413/figures/color_feh_yfe_bafe.jpg', 'tmp_2509.13413/figures/color_feh_yfe_ecc.jpg', 'tmp_2509.13413/figures/color_feh_yfe_mwd.jpg']
copying  tmp_2509.13413/figures/P_e_general.jpg to _build/html/
copying  tmp_2509.13413/figures/color_feh_yfe_bafe.jpg to _build/html/
copying  tmp_2509.13413/figures/color_feh_yfe_ecc.jpg to _build/html/
copying  tmp_2509.13413/figures/color_feh_yfe_mwd.jpg to _build/html/
exported in  _build/html/2509.13413.md
    + _build/html/tmp_2509.13413/figures/P_e_general.jpg
    + _build/html/tmp_2509.13413/figures/color_feh_yfe_bafe.jpg
    + _build/html/tmp_2509.13413/figures/color_feh_yfe_ecc.jpg
    + _build/html/tmp_2509.13413/figures/color_feh_yfe_mwd.jpg
found figures ['tmp_2509.13587/./astrometric_orbit_J1446.png', 'tmp_2509.13587/./RV_OC_J1446_InstAll.png', 'tmp_2509.13587/./J1446_lightcurve-SPOC.png', 'tmp_2509.13587/./cpmtest_J1446.png']
copying  tmp_2509.13587/./astrometric_orbit_J1

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\noun}[1]{\textsc{\MakeLowercase{#1}}}$
$\newcommand{\code}{\texttt}$
$\newcommand{\m}[1]{M_#1}$
$\newcommand{\mwd}{M_\text{WD}}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\bah}{[Ba/H]}$
$\newcommand{\bafe}{[Ba/Fe]}$
$\newcommand{\yh}{[Y/H]}$
$\newcommand{\yfe}{[Y/Fe]}$
$\newcommand{\sH}{[s/H]}$
$\newcommand{\sfe}{[s/Fe]}$
$\newcommand{\rl}{r\textsubscript{L}}$
$\newcommand{\rorb}{R\textsubscript{orb}}$
$\newcommand{\sproc}{\textit{s}-process}$</div>



<div id="title">

# Gaia Barium Dwarfs and Their Ostensibly Ordinary Counterparts

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.13413-b31b1b.svg)](https://arxiv.org/abs/2509.13413)<mark>Appeared on: 2025-09-18</mark> - 

</div>
<div id="authors">

P. Rekhi, et al. -- incl., <mark>S. Shahaf</mark>, <mark>J. Müller-Horn</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** The recently identified Gaia population of main-sequence--white dwarf (MS+WD) binaries at separations of ${\sim} 1 {\rm AU}$ , often with moderate eccentricities, is not readily reproduced by binary population synthesis models.Barium stars represent a closely related population whose enrichment in $\sproc$ elements both confirms the presence of a WD companion and attests to past binary interaction.It also indicates that mass transfer occurred at least during the late and post-AGB phases of the WD progenitor, when $\sproc$ elements are dredged up.In this work, we further explore the connection between the astrometrically identified Gaia MS+WD binaries and the classical barium star population.To achieve this, we used high-resolution FEROS spectroscopy to measure abundances for 30 Gaia DR3 non-single-star binaries, identifying 10 as Ba-enriched. Together with our recent analysis of archival GALAH data, this yields a sample of 38 barium dwarfs with dynamically measured WD masses, compared to only 6 previously known systems with known WD masses at these separations.We find that, in cases where metallicity is sufficiently low to facilitate efficient $\sproc$ production, barium and yttrium enrichment is often detected. This enrichment is also identified in eccentric systems, suggesting that post-AGB mass transfer mechanisms are capable of pumping eccentricity into the orbit or occur without erasing it.Our results indicate that the Gaia MS+WD binaries trace the population from which barium stars emerge.  Treating the large Gaia-discovered population as an extension of known $\sproc$ enriched dwarfs opens an avenue to empirically constrain their formation and evolution.

</div>

<div id="div_fig1">

<img src="tmp_2509.13413/figures/P_e_general.jpg" alt="Fig2" width="100%"/>

**Figure 2. -** Period-Eccentricity diagram of the Shahaf_2024_TriageGaia AMRF sample (gray), its Ba-enriched subset from this work and \citetalias{Rekhi_2024_BaEnrichment}(red) and Ba/CH/CEMP-s stars from literature \citep[green/blue; ][]{Jorissen_2016_BinaryProperties, Escorza_2019_BariumRelated, Escorza_2023_BariumRelated}. The samples show scatter consistent with each other. Our samples uncover a significantly larger population of Ba dwarfs at $e$ > 0.1 at P < 1000 d, which is underrepresented in literature. The AMRF sample thus provides a systematic route to identifying large samples of Ba dwarfs with orbital parameters and companion WD masses from the Gaia NSS catalog. (*fig:P e general*)

</div>
<div id="div_fig2">

<img src="tmp_2509.13413/figures/color_feh_yfe_bafe.jpg" alt="Fig7" width="100%"/>

**Figure 7. -** [Ba/Fe] vs [Y/Fe] coloured by [Fe/H], for the 10 targets having Y measurements. The green dashed line denotes the enrichment boundary.
Ba-enhanced stars generally exhibit Y enhancement, reducing the likelihood of spurious Ba/Y enhancement.  (*fig:ba y comp*)

</div>
<div id="div_fig3">

<img src="tmp_2509.13413/figures/color_feh_yfe_ecc.jpg" alt="Fig8.1" width="50%"/><img src="tmp_2509.13413/figures/color_feh_yfe_mwd.jpg" alt="Fig8.2" width="50%"/>

**Figure 8. -** [Y/Fe] as function of (a) eccentricity and (b) $\m$wd. (*fig: Y color plots*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.13413"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\uat}[2]{\href{http://vocabs.ands.org.au/repository/api/lda/aas/the-unified-astronomy-thesaurus/current/resource.html?uri=http://astrothesaurus.org/uat/#1}{#2 (#1)}}$
$\newcommand\mk{#1}$</div>



<div id="title">

# Direct Imaging Explorations for Companions from the Subaru/IRD Strategic Program II; Discovery of a Brown-dwarf Companion around a Nearby mid-M dwarf LSPM J1446+4633

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.13587-b31b1b.svg)](https://arxiv.org/abs/2509.13587)<mark>Appeared on: 2025-09-18</mark> -  _11 pages, 7 figs, 4 tables, accepted for publication in AJ_

</div>
<div id="authors">

T. Uyama, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of a new directly-imaged brown dwarf companion with Keck/NIRC2+pyWFS around a nearby mid-type M dwarf LSPM J1446+4633 (hereafter J1446). The $L'$ -band contrast ( $4.5\times10^{-3}$ ) is consistent with a $\sim20-60 M_{\rm Jup}$ object at 1--10 Gyr and our two-epoch NIRC2 data suggest a $\sim30\%$ ( $\sim3.1\sigma)$ variability in its $L'$ -band flux. We incorporated Gaia DR3 non-single-star catalog into the orbital fitting by combining the Subaru/IRD RV monitoring results, NIRC2 direct imaging results, and Gaia proper motion acceleration. As a result, we derive ${59.8}_{-1.4}^{+1.5} M_{\rm Jup}$ and $\approx4.3 {\rm au}$ for the dynamical mass and the semi-major axis of the companion J1446B, respectively.J1446B is one of the intriguing late-T dwarfs showing variability at $L'$ -band for future atmospheric studies with the constrained dynamical mass.Because the J1446 system is accessible with various observation techniques such as astrometry, direct imaging, and high-resolution spectroscopy including radial velocity measurement, it has a potential as a great benchmark system to improve our understanding for cool dwarfs.

</div>

<div id="div_fig1">

<img src="tmp_2509.13587/./astrometric_orbit_J1446.png" alt="Fig4.1" width="50%"/><img src="tmp_2509.13587/./RV_OC_J1446_InstAll.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** 
    Left: Modeled projected orbits of J1446B from fitting the Gaia DR3 acceleration (a red arrow in left) and two measured Keck/NIRC2 relative astrometric measurements (blue dots in left), and Subaru/IRD RVs (red dots in right). The host star defines the origin and is indicated by a black star; one hundred random orbital draws are shown and color-coded by companion mass, with the best-fit orbit (black). The fitted offset at 2016.0 is plotted in red, and the direction of the measured Gaia DR3 acceleration is indicated by the red arrow. Right: Modeled RVs from a random sampling of orbits from the posterior are color-coded by companion mass with the best-fit orbit.
     (*fig: orvara fitting*)

</div>
<div id="div_fig2">

<img src="tmp_2509.13587/./J1446_lightcurve-SPOC.png" alt="Fig3" width="100%"/>

**Figure 3. -** TESS lightcurves of J1446 ranging four sectors. Left: Comparisons of the measured flux variations in the four sectors, where we adopted median time of each section as a reference date. Right: The same comparison as the left panel except for the y-axis with normalized fluxes. (*fig: TESS*)

</div>
<div id="div_fig3">

<img src="tmp_2509.13587/./cpmtest_J1446.png" alt="Fig1" width="100%"/>

**Figure 1. -** Common proper motion test of the companion candidate around J1446. The blue and orange crosses indicate the relative location to the central star ($\Delta{\rm RA}$ for x-axis and $\Delta{\rm Dec}$ for y-axis, respectively). The gray solid line indicates a trajectory of a zero proper-motion background star and the red cross indicates an expected position of the background-star case in the second epoch. We insert the zoomed-in panel for the relative astrometry of the first and second epochs, whose error-bars are small in the original panel.  (*fig: cpm test*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.13587"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\SII}{[S~{\sc ii}]}$
$\newcommand{\SIII}{[S~{\sc iii}]}$
$\newcommand{\OIII}{[O~{\sc iii}]}$
$\newcommand{\NII}{[N~{\sc ii}]}$
$\newcommand{\HII}{H~{\sc ii}}$
$\newcommand{\HeII}{He~{\sc ii}}$
$\newcommand{\HI}{H~{\sc i}}$
$\newcommand{\Ha}{H\alpha}$
$\newcommand{\Hb}{H\beta}$
$\newcommand{\kms}{ \mbox{km} \mbox{s}^{-1}}$
$\newcommand{\ergs}{ \mbox{erg} \mbox{s}^{-1}}$
$\newcommand{\HST}{\textit{HST}}$
$\newcommand{\JWST}{\textit{JWST}}$
$\newcommand{\SIIHa}{[S~{\sc ii}]/H\alpha}$
$\newcommand{\SIIISII}{[S~{\sc iii}]/[S~{\sc ii}]}$
$\newcommand{\NIIHa}{[N~{\sc ii}]/H\alpha}$
$\newcommand{\OIIIHb}{[O~{\sc iii}]/H\beta}$
$\newcommand{\RPAH}{{\rm R_{PAH}}}$
$\newcommand{\RPAHst}{{\rm R_{PAH}^*}}$
$\newcommand{\SBDIG}{\mathrm{\Sigma(H\alpha)_{DIG}}}$
$\newcommand{\sigmaHa}{\sigma(H\alpha)}$
$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\green}{\textcolor{green}}$
$\newcommand{\note}{\red}$
$\newcommand{\revone}\begin{document}$
$\newcommand{\numHII}{17151}$
$\newcommand{\numSNRs}{\revone{396}}$
$   \title{Polycyclic aromatic hydrocarbons destruction in star-forming regions across 42 nearby galaxies}$
$    \titlerunning{PHANGS-JWST and MUSE: PAHs in star-forming regions}$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IPAC}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{\Carnegie}{\label{Carnegi} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{ÇAPP}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\CfA}{\label{CfA} Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\CITEVA}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CNRS}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\HD}{\label{HD} Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\ICRAR}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{\IRAM}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{\ITA}{\label{ITA} Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\IWR}{\label{IWR} Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\JHU}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\Leiden}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\MPE}{\label{MPE} Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\Nagoya}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{\NRAO}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ObsParis}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}$
$\newcommand{\UToledo}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{\Toulouse}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\UChile}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\UConn}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\UCSD}{\label{UCSD} Department of Astronomy \& Astrophysics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\ F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UMass}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LAM}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{\UHawaii}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UCM}{\label{UCM} Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\IPARC}{\label{IPARC} Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\McMaster}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\INAF}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}$
$\newcommand{\Sydney}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{\UA}{\label{UA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CITA}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ASIAA}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 106319, Taiwan}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\PSMA}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{\ILL}{\label{ILL} ILL}$
$\newcommand{\Whitman}{\label{Whitman} Whitman College, 345 Boyer Avenue, Walla Walla, WA 99362, USA}$
$\newcommand{\Ox}{\label{Ox} Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\stromlo}{\label{stromlo} Research School of Astronomy and Astrophysics, Australian National University, Mt Stromlo Observatory, Weston Creek, ACT 2611, Australia}$
$\newcommand{\FAU}{\label{FAU} Dr. Karl-Remeis Sternwarte and Erlangen Centre for Astroparticle Physics, Friedrich-Alexander Universitat Erlangen-Nurnberg, Sternwartstr. 7, 96049 Bamberg, Germany}$
$\newcommand{\Nice}{\label{Nice} Université C\^ote d'Azur, Observatoire de la C\^ote d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}$
$\newcommand{\Gemini}{\label{Gemini} International Gemini Observatory, NSF NOIRLab, 950 N Cherry Ave, Tucson, AZ 85719, USA}$
$\newcommand{\ESOchile}{\label{ESOchile} European Southern Observatory (ESO), Alonso de Córdova 3107, Casilla 19, Santiago 19001, Chile}$
$\newcommand{\UNAM}{\label{UNAM}$
$Instituto de Astronomía, Universidad Nacional Autónoma de México, Ap. 70-264, 04510 CDMX, Mexico}$
$\newcommand{\KIPAC}{\label{KIPAC} Kavli Institute for Particle Astrophysics \& Cosmology, Stanford University, CA 94305, USA}$
$\newcommand{\CDU}{\label{CDU} Center for Decoding the Universe, Stanford University, CA 94305, USA}$
$\newcommand{\UnknownInst}{\label{UnknownInst} What is your affiliation?}$
$\author{$
$        Oleg V. Egorov\orcid{0000-0002-4755-118X}\inst{\ref{HD}}\thanks{\email{oleg.egorov@uni-heidelberg.de}}$
$        \and$
$    Adam~K.~Leroy\orcid{0000-0002-2545-1700}$
$\inst{\ref{OSU}, \ref{CCAPP}}$
$\and$
$    Karin Sandstrom\orcid{0000-0002-4378-8534}\inst{\ref{UCSD}}$
$    \and$
$    Kathryn Kreckel\orcid{0000-0001-6551-3091}\inst{\ref{HD}}$
$    \and$
$    Dalya Baron\orcid{0000-0003-4974-3481}\inst{\ref{KIPAC}, \ref{CDU}}$
$    \and$
$    Francesco Belfiore\orcid{0000-0002-2545-5752}\inst{\ref{INAF}}$
$       \and$
$    Ryan Chown\orcid{0000-0001-8241-7704}\inst{\ref{OSU}}$
$    \and$
$    Jessica Sutter\orcid{0000-0002-9183-8102}\inst{\ref{Whitman}}$
$    \and$
$   Médéric Boquien\orcid{0000-0003-0946-6176}$
$\inst{\ref{Nice}}$
$\and$
$    Mar Canal i Saguer\inst{\ref{FAU}}$
$    \and$
$    Enrico Congiu\orcid{0000-0002-8549-4083}\inst{\ref{ESOchile}}$
$    \and$
$Daniel~A.~Dale\orcid{0000-0002-5782-9093}\inst{\ref{UWyoming}}$
$    \and$
$    Evgeniya Egorova\orcid{0000-0003-2717-8784}\inst{\ref{HD}}$
$    \and$
$    Michael Huber\inst{\ref{HD}}$
$    \and$
$    Jing Li\orcid{0000-0002-4825-9367}\inst{\ref{HD}}$
$   \and$
$Thomas~G.~Williams\orcid{0000-0002-0012-2142}$
$    \inst{\ref{Ox}}$
$    \and$
$    Jérémy Chastenet\orcid{0000-0002-5235-5589}\inst{\ref{UGent}}$
$    \and$
$    I-Da Chiang\orcid{0000-0003-2551-7148}\inst{\ref{ASIAA}}$
$    \and$
$    Ivan Gerasimov\orcid{0000-0001-7113-8152}\inst{\ref{Nice}}$
$    \and$
$    Hamid Hassani\orcid{0000-0002-8806-6308}\inst{\ref{Alberta}}$
$\and$
$   Hwihyun Kim\orcid{0000-0003-4770-688X}\inst{\ref{Gemini}}$
$   \and$
$   Hannah Koziol\orcid{0009-0001-5949-1524}\inst{\ref{UCSD}}$
$   \and$
$   Janice C. Lee\orcid{0000-0003-0946-6176}\inst{\ref{STScI}}$
$   \and$
$   Rebecca L. McClain\orcid{0000-0002-6187-4866}\inst{\ref{OSU}, \ref{CCAPP}}$
$\and$
$José Eduardo Méndez Delgado\orcid{0000-0002-6972-6411}\inst{\ref{UNAM}}$
$   \and$
$Hsi-An Pan\orcid{0000-0002-1370-6964}\inst{\ref{TKU}}$
$\and$
$   Debosmita Pathak\orcid{0000-0003-2721-487X$
$}\inst{\ref{OSU}}$
$    \and$
$    Erik Rosolowsky\orcid{0000-0002-5204-2259}\inst{\ref{Alberta}}$
$    \and$
$    Sumit K. Sarbadhicary\orcid{0000-0002-4781-7291}$
$\inst{\ref{OSU}, \ref{CCAPP}}$
$\and$
$    Eva Schinnerer\orcid{0000-0002-3933-7677}\inst{\ref{MPIA}}$
$    \and$
$    David Thilker\orcid{0000-0002-8528-7340}\inst{\ref{JHU}}$
$    \and$
$    Leonardo Ubeda\orcid{0000-0001-7130-2880}\inst{\ref{STScI}}$
$    \and$
$    Tony Weinbeck\orcid{0009-0005-8923-558X}\inst{\ref{UWyoming}}$
$}$
$\institute{\HD \and \OSU \and ÇAPP \and \UCSD  \and \KIPAC \and \CDU \and \INAF  \and \Whitman  \and  \Nice \and \FAU  \and \ESOchile \and \UWyoming \and \Ox   \and \UGent \and \ASIAA \and \Alberta \and  \Gemini \and \STScI \and \UNAM \and \TKU \and \MPIA \and \JHU$
$}$
$   \date{Received ????? ??, 2025; accepted ????? ??, ????}$
$  \abstract{$
$  Polycyclic aromatic hydrocarbons (PAHs) are widespread in the interstellar medium (ISM) of \simSolar metallicity galaxies, where they play a critical role in ISM heating, cooling, and reprocessing stellar radiation. The PAH fraction, the abundance of PAHs relative to total dust mass, is a key parameter in ISM physics. Using JWST and MUSE observations of 42 galaxies from the PHANGS survey, we analyze the PAH fraction in over 17 000 \HII regions spanning a gas-phase oxygen abundance of \rm 12+\log(O/H) = 8.0–8.8 (Z \sim 0.2–1.3~Z_\odot), and \sim400 isolated supernova remnants (SNRs). We find a significantly lower PAH fraction \revone{toward} \HII regions compared to a reference sample of diffuse ISM areas at matched metallicity. At \rm 12+\log(O/H) > 8.2, the PAH fraction \revone{toward} \HII regions is strongly anti-correlated with the local ionization parameter, suggesting that PAH destruction is correlated with ionized gas and/or hydrogen-ionizing UV radiation.$
$  At lower metallicities, the PAH fraction declines steeply in both \HII regions and the diffuse ISM, likely reflecting less efficient PAH formation in metal-poor environments. Carefully isolating dust emission from the vicinity of optically-identified supernova remnants, we see evidence for selective PAH destruction from measurements of lower PAH fractions, which is, however, indistinguishable at \sim 50 pc scales.$
$  Overall, our results point to ionizing radiation as the dominant agent of PAH destruction within \HII regions, with metallicity playing a key role in their global abundance in galaxies.$
$}$
$   \keywords{Galaxies: ISM --$
$                Infrared: ISM -- ISM: HII regions -- ISM: dust, extinction -- ISM: abundances$
$               }$
$   \maketitle$
$\n\end{equation}\end{document}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\numHII}{17151}$
$\newcommand{\numSNRs}{\revone{396}}$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IPAC}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{\Carnegie}{\label{Carnegi} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{ÇAPP}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\CfA}{\label{CfA} Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\CITEVA}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CNRS}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\HD}{\label{HD} Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\ICRAR}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{\IRAM}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{\ITA}{\label{ITA} Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\IWR}{\label{IWR} Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\JHU}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\Leiden}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\MPE}{\label{MPE} Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\Nagoya}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{\NRAO}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ObsParis}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}$
$\newcommand{\UToledo}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{\Toulouse}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\UChile}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\UConn}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\UCSD}{\label{UCSD} Department of Astronomy \& Astrophysics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\ F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UMass}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LAM}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{\UHawaii}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UCM}{\label{UCM} Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\IPARC}{\label{IPARC} Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\McMaster}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\INAF}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}$
$\newcommand{\Sydney}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{\UA}{\label{UA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CITA}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ASIAA}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 106319, Taiwan}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\PSMA}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{\ILL}{\label{ILL} ILL}$
$\newcommand{\Whitman}{\label{Whitman} Whitman College, 345 Boyer Avenue, Walla Walla, WA 99362, USA}$
$\newcommand{\Ox}{\label{Ox} Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\stromlo}{\label{stromlo} Research School of Astronomy and Astrophysics, Australian National University, Mt Stromlo Observatory, Weston Creek, ACT 2611, Australia}$
$\newcommand{\FAU}{\label{FAU} Dr. Karl-Remeis Sternwarte and Erlangen Centre for Astroparticle Physics, Friedrich-Alexander Universitat Erlangen-Nurnberg, Sternwartstr. 7, 96049 Bamberg, Germany}$
$\newcommand{\Nice}{\label{Nice} Université C\^ote d'Azur, Observatoire de la C\^ote d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}$
$\newcommand{\Gemini}{\label{Gemini} International Gemini Observatory, NSF NOIRLab, 950 N Cherry Ave, Tucson, AZ 85719, USA}$
$\newcommand{\ESOchile}{\label{ESOchile} European Southern Observatory (ESO), Alonso de Córdova 3107, Casilla 19, Santiago 19001, Chile}$
$\newcommand{\UNAM}{\label{UNAM}$
$Instituto de Astronomía, Universidad Nacional Autónoma de México, Ap. 70-264, 04510 CDMX, Mexico}$
$\newcommand{\KIPAC}{\label{KIPAC} Kavli Institute for Particle Astrophysics \& Cosmology, Stanford University, CA 94305, USA}$
$\newcommand{\CDU}{\label{CDU} Center for Decoding the Universe, Stanford University, CA 94305, USA}$
$\newcommand{\UnknownInst}{\label{UnknownInst} What is your affiliation?}$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\green}{\textcolor{green}}$
$\newcommand{\note}{\red}$
$\newcommand{\revone}\begin$</div>



<div id="title">

# Polycyclic aromatic hydrocarbons destruction in star-forming regions across 42 nearby galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.13845-b31b1b.svg)](https://arxiv.org/abs/2509.13845)<mark>Appeared on: 2025-09-18</mark> -  _19 pages, 15 figures. Accepted for publication in A&A_

</div>
<div id="authors">

O. V. Egorov, et al. -- incl., <mark>K. Kreckel</mark>, <mark>J. Li</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Polycyclic aromatic hydrocarbons (PAHs) are widespread in the interstellar medium (ISM) of $\sim$ Solar metallicity galaxies, where they play a critical role in ISM heating, cooling, and reprocessing stellar radiation. The PAH fraction, the abundance of PAHs relative to total dust mass, is a key parameter in ISM physics. Using JWST and MUSE observations of 42 galaxies from the PHANGS survey, we analyze the PAH fraction in over 17 000 $\HII$ regions spanning a gas-phase oxygen abundance of $\rm 12+\log(O/H) = 8.0$ –8.8 ( $Z \sim 0.2$ –1.3 $Z_\odot$ ), and $\sim400$ isolated supernova remnants (SNRs). We find a significantly lower PAH fraction $\revone{toward}$ $\HII$ regions compared to a reference sample of diffuse ISM areas at matched metallicity. At $\rm 12+\log(O/H) > 8.2$ , the PAH fraction $\revone{toward}$ $\HII$ regions is strongly anti-correlated with the local ionization parameter, suggesting that PAH destruction is correlated with ionized gas and/or hydrogen-ionizing UV radiation.  At lower metallicities, the PAH fraction declines steeply in both $\HII$ regions and the diffuse ISM, likely reflecting less efficient PAH formation in metal-poor environments. Carefully isolating dust emission from the vicinity of optically-identified supernova remnants, we see evidence for selective PAH destruction from measurements of lower PAH fractions, which is, however, indistinguishable at $\sim 50$ pc scales.  Overall, our results point to ionizing radiation as the dominant agent of PAH destruction within $\HII$ regions, with metallicity playing a key role in their global abundance in galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2509.13845/./figs/RPAH_vs_ip_v20_rev1.png" alt="Fig12.1" width="50%"/><img src="tmp_2509.13845/./figs/RPAH_vs_ip_v20_resolved_rev1.png" alt="Fig12.2" width="50%"/>

**Figure 12. -** PAH fraction anti-correlates with $\log$($\SII$I/$\SII$) (ionization parameter tracer) for thousands $\HII$ regions in 42 PHANGS-JWST galaxies. Panel (a, b) shows all $\sim 17200$ analyzed $\HII$ regions, while  $\sim 5600$ brightest \revone{"resolved"}$\HII$ regions are shown on Panels (c, d). The dashed red line shows the best-fit linear regression to the \revone{"resolved"} sample defined by Eq. \ref{eq:RPAH_vs_ip}. Histograms on panels (b, d) show how the logarithmic residuals after subtracting this linear trend depend on metallicity. Color on panels (a, c) denotes gas-phase oxygen abundance; red contours show the probability density of the high-metallicity ($\rm 12+\log(O/H) > 8.2$) points (levels correspond to 50, 65, 80, 95 and 99 percentile intervals). The Spearman correlation coefficient ($\rho$) and RMS scatter around the linear fit are reported on panels (a, c).  (*fig:RPAH_vs_ip*)

</div>
<div id="div_fig2">

<img src="tmp_2509.13845/./figs/hist_diff_hii_rpah7_v20_rev1.png" alt="Fig4" width="100%"/>

**Figure 4. -** Distribution of $\RPAH$st for \revone{"resolved"}(luminous) $\HII$ regions (orange) and the regions of the similar size from diffuse ISM (blue) for several gas-phase metallicity bins. Dashed and two dotted lines on each distribution show median and 25 and 75 percentile quartiles, respectively. Both $\RPAH$ and $\RPAH$st are lower in $\HII$ regions than in diffuse ISM. A strong decrease of $\RPAH$st in the lowest metallicity bin is seen for both $\HII$ regions and diffuse ISM. (*fig:diff_vs_hii*)

</div>
<div id="div_fig3">

<img src="tmp_2509.13845/./figs/RPAH_vs_met_v2p0_rev1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Distribution of $\rm \log({\rm R_{PAH}^*})$ vs oxygen abundance color-coded by $\rm \log([SIII]/[SII])$(tracer of the ionization parameter) for all $\HII$ regions in the sample. Red circles show average values in different metallicity bins (reported in Tab. \ref{tab:Rpah_vs_z_bins}). Vertical gray dashed line marks $\rm 12 + \log(O/H) = 8.2$ as a threshold between the low- and high-metallicity regimes. The horizontal dashed line correspond to the average $\RPAH$ for diffuse ISM measured by [Sutter, Sandstrom and Chastenet (2024)](). Black squares correspond to measurements for WLM and NGC6822 (from low- to high-metallicity) made by  ([Chown, Leroy and Bolatto 2025]())  using JWST F770W and F2100W images. The PAH fraction drops significantly in the low-metallicity regime.  (*fig:RPAH_vs_met*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.13845"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\civ}{C{ \tiny IV}}$
$\newcommand{\mgii}{Mg{ \tiny II}}$
$\newcommand{\oi}{O{ \tiny I}}$
$\newcommand{\oiii}{[O{ \tiny III}]}$
$\newcommand{\feii}{Fe{ \tiny II}}$
$\newcommand{\hi}{H{ \tiny I}}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\hg}{H\gamma}$
$\newcommand{\rfe}{\mathcal{R}_\mathrm{Fe}}$</div>



<div id="title">

# Spatially resolved broad line region in a quasar at z=4: Dynamical black hole mass and prominent outflow

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.13911-b31b1b.svg)](https://arxiv.org/abs/2509.13911)<mark>Appeared on: 2025-09-18</mark> -  _submitted to A&A (15 pages, 10 figures)_

</div>
<div id="authors">

G. Collaboration, et al. -- incl., <mark>W. Brandner</mark>, <mark>P. Garcia</mark>, <mark>S. Joharle</mark>, <mark>L. Kreidberg</mark>, <mark>J. Sauter</mark>

</div>
<div id="abstract">

**Abstract:** We present the first near-infrared interferometric data of a QSO at z=4. The K-band observations were performed with GRAVITY+ on the VLTI using all 4 UTs, detecting a differential phase signal that traces the spatially resolved kinematics for both the $\hb$ and $\hg$ lines in the broad line region. We fit the two lines simultaneously with an updated model that includes distinct rotating and conical outflowing components. We find that more than 80 \% of the $\hi$ line emission from the BLR originates in an outflow with a velocity up to $10^4$ km s $^{-1}$ . This is oriented so that our line of sight is along an edge of the conical structure, which produces the prominent blue wing on the line profile. A combination of anisotropic line emission and mid-plane opacity lead to the single-sided phase signal. The model is able to qualitatively match both the outflowing $\civ$ line profile and the systemic $\oi$ fluorescent emission. The derived black hole mass of $8\times10^8$ M $_\odot$ is the highest redshift black hole mass measurement to date obtained directly from BLR dynamics. It is an order of magnitude lower than that inferred from various single epoch scaling relations, and implies that the accretion is highly super-Eddington. With reference to recent simulations, the data suggest that this QSO is emitting close to its radiative limit in a regime where strong outflows are expected around a polar conical region.

</div>

<div id="div_fig1">

<img src="tmp_2509.13911/./GRAVITY_avg_phase.png" alt="Fig6.1" width="50%"/><img src="tmp_2509.13911/./GRAVITY_outflow_vs_disk.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Flux spectrum, averaged differential phase spectra, and line decomposition.
The top row shows the continuum subtracted spectrum of the two emission lines, and the best fit model.
For each of the $\hb$(left) and $\hg$(right) lines, the middle row shows the average phase spectra  which are created from the three baselines with the strongest signal (this is for visualisation purposes only; phase spectra for the individual baselines are shown in Fig. \ref{fig:allphases}).
The bottom row shows the decomposition of the fitted line profile (red) into the outflow (green) and disky (blue) components. The outflow dominates the total line flux, including much of the apparent core of the line profile. (*fig:avgphase*)

</div>
<div id="div_fig2">

<img src="tmp_2509.13911/./specJ0529s_fit.png" alt="Fig4" width="100%"/>

**Figure 4. -** K-band spectrum of J0529 from ERIS. The upper panel shows the data (blue; the narrow features at the long end of the band are due to imperfect atmospheric correction) and the fit (orange), with the $\hi$ lines marked (the location of the $\oi$ii lines are also marked; the 4363 Å line is expected to be 10-100 times weaker than those at 4959 Å and 5007 Å, the peaks of which are out of the spectral range). Also shown is the relative atmospheric transmission and the residual, indicating which regions have been excluded from the fit.
The lower panel shows the components of the fit: the $\hi$ lines (black) and $\feii$ complex (red), as well as a component that represents either poor slope correction at the band edge (as is seen also at the short end of the band) or a possible blue wing on the $\oi$ii line. (*fig:erisspec*)

</div>
<div id="div_fig3">

<img src="tmp_2509.13911/./GRAVITY_line_comparison.png" alt="Fig5" width="100%"/>

**Figure 5. -** Comparison of $\hi$ profile to $\civ$$\lambda$1549 Å (left) and $\oi$$\lambda$1302 Å (right) in the Xshooter spectrum \citep{wol24}. The normalisations are independent, and the velocity zero point here is defined by the $\hi$ line peak rather than the true systemic velocity.
$\civ$ is associated only with the BLR outflow, and extends to higher velocities.
$\oi$ is more associated with the static rotating part of the BLR (the source of the deep absorption at multiple velocities across the line peak is unclear). (*fig:linecomp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.13911"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\mnras}{MNRAS}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\jqsrt}{JQSRT}$
$\newcommand{\aj}{AJ}$
$\newcommand{\apj}{ApJ}$
$\newcommand{\apjl}{ApJL}$
$\newcommand{\apjs}{ApJS}$
$\newcommand{\aap}{A\&Aj}$
$\newcommand{\araa}{ARAA}$
$\newcommand{\nat}{Nature}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\vect}[1]{\mathbf{#1}}$
$\newcommand{\mt}[1]{\mathrm{#1}}$
$\newcommand{\Msun}{\ensuremath{ M_{\odot}}}$
$\newcommand{\Mearth}{\ensuremath{ M_{\oplus}}}$
$\newcommand{\Mmoon}{\ensuremath{ M_{\mathrm{Moon}}}}$
$\newcommand{\Mjup}{\ensuremath{ M_{\mathrm{Jup}}}}$
$\newcommand{\Rjup}{\ensuremath{ R_{\mathrm{Jup}}}}$
$\newcommand{\mum}{\ensuremath{\mathrm{ \mu m}}}$
$\newcommand{\AMMF}{\ensuremath{\mathrm{AMMF}}}$
$\newcommand{\Zatm}{\ensuremath{Z_{\mathrm{atm}}}}$
$\newcommand{\Zenv}{\ensuremath{Z_{\mathrm{env}}}}$
$\newcommand{\farcmin}{\mbox{\ensuremath{.\mkern-4mu^\prime}}}$
$\newcommand{\farcsec}{\mbox{\ensuremath{ .\!\!^{\prime\prime}}}}$
$\newcommand{\water}{H_{2}O}$
$\newcommand{\logX}[1]{\ensuremath{\log(\mathrm{X_{\ce{#1}}})}}$
$\newcommand{\logXratio}[2]{\ensuremath{\log(\mathrm{X_{\ce{#1}} / X_{\ce{#2}} })}}$
$\newcommand{\rstar}{R_*}$
$\newcommand{\re}{ R_\oplus}$
$\newcommand{\me}{ M_\oplus}$
$\newcommand{\rsun}{ R_\odot}$
$\newcommand{\project}[1]{\textsl{#1}}$
$\newcommand{\JWST}{\project{JWST}}$
$\newcommand{\HST}{\project{HST}}$
$\newcommand{\Hubble}{\project{Hubble}}$
$\newcommand{\Spitzer}{\project{Spitzer}}$
$\newcommand{\Kepler}{\project{Kepler}}$
$\newcommand{\Ktwo}{\project{K2}}$
$\newcommand{\TESS}{\project{TESS}}$
$\newcommand{\CHEOPS}{\project{CHEOPS}}$
$\newcommand$
$\newcommand$
$\newcommand{\umontreal}{Department of Physics and Trottier Institute for Research on Exoplanets, Université de Montréal, Montreal, QC, Canada}$
$\newcommand{\ucla}{Department of Earth, Planetary, and Space Sciences, University of California, Los Angeles, Los Angeles, CA, USA}$
$\newcommand{\uwashington}{Department of Earth and Space Sciences/Astrobiology Program, University of Washington, Seattle, WA, USA}$
$\newcommand{\uchicago}{Department of Astronomy \& Astrophysics, University of Chicago, Chicago, IL, USA}$
$\newcommand{\mpia}{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\newcommand{\sron}{SRON Netherlands Institute for Space Research, Niels Bohrweg 4, 2333 CA Leiden, The Netherlands}$
$\newcommand{\leiden}{Leiden Observatory, Leiden University, Einsteinweg 55, 2333 CC Leiden, The Netherlands}$
$\newcommand{\cab}{Centro de Astrobiología (CAB), CSIC-INTA, ESAC campus, Camino Bajo del Castillo s/n, 28692, Villanueva de la Cañada (Madrid), Spain}$
$\newcommand{\porto}{Instituto de Astrofísica e Ciências do Espaço, Universidade do Porto, CAUP, Rua das Estrelas, 4150-762 Porto, Portugal}$
$\newcommand{\uclaAstro}{Department of Physics \& Astronomy, University of California Los Angeles, Los Angeles, CA 90095, USA}$
$\newcommand{\uwisconsin}{Department of Astronomy, University of Wisconsin--Madison, Madison, WI 53706, USA}$
$\newcommand{\mcmaster}{Department of Physics \& Astronomy, McMaster University, Hamilton, ON, Canada}$
$\newcommand{\oxford}{Department of Physics, University of Oxford, Parks Rd, Oxford, OX1 3PU, UK}$
$\newcommand{\heidelberg}{Department of Physics and Astronomy, Heidelberg University, Im Neuenheimer Feld 226, D-69120 Heidelberg, Germany}$
$\newcommand{\riogrande}{Departamento de Física Teórica e Experimental, Universidade Federal do Rio Grande do Norte, Campus Universitário, Natal, RN, 59072-970, Brazil}$
$\newcommand{\geneve}{Observatoire de Genève, Département d’Astronomie, Université de Genève, Chemin Pegasi 51, 1290 Versoix, Switzerland}$
$\newcommand{\newcastle}{School of Information and Physical Sciences, University of Newcastle, Callaghan, NSW, Australia}$
$\newcommand{\grenoble}{Univ. Grenoble Alpes, CNRS, IPAG, 38000 Grenoble, France}$
$\newcommand{\iac}{Instituto de Astrofísica de Canarias, c/ Vía Láctea s/n, 38205$
$La Laguna, Tenerife, Spain\label{iac}$
$Departamento de Astrofísica, Universidad de La Laguna, 38206 La Laguna, Tenerife, Spain}$</div>



<div id="title">

# $\large{Possible Evidence for the Presence of Volatiles on the Warm Super-Earth TOI-270 b}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.14224-b31b1b.svg)](https://arxiv.org/abs/2509.14224)<mark>Appeared on: 2025-09-18</mark> -  _Published in The Astronomical Journal_

</div>
<div id="authors">

L.-P. Coulombe, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>C. Gapp</mark>

</div>
<div id="abstract">

**Abstract:** The search for atmospheres on rocky exoplanets is a crucial step in understanding the processes driving atmosphere formation, retention, and loss.Past studies have revealed the existence of planets interior to the radius valley with densities lower than would be expected for pure-rock compositions, indicative of the presence of large volatile inventories which could facilitate atmosphere retention.Here we present an analysis of the JWST NIRSpec/G395H transmission spectrum of the warm ( $T_\mathrm{eq,A_B=0} = 569$ K) super-Earth TOI-270 b ( $R_\mathrm{p}$ = 1.306 $R_\oplus$ ), captured alongside the transit of TOI-270 d. The JWST white light-curve transit depth updates TOI-270 b's density to $\rho_\mathrm{p}$ = $3.7\pm0.5$ g/cm $^3$ , inconsistent at 4.4 $\sigma$ with an Earth-like composition. Instead, the planet is best explained by a non-zero, percent-level water mass fraction, possibly residing on the surface or stored within the interior.The JWST transmission spectrum shows possible spectroscopic evidence for the presence of this water as part of an atmosphere on TOI-270 b, favoring a $H_2$ O-rich steam atmosphere model over a flat spectrum ( $\ln\mathcal{B}=0.3-3.2$ , inconclusive to moderate), with the exact significance depending on whether an offset parameter between the NIRSpec detectors is included. We leverage the transit of the twice-larger TOI-270 d crossing the stellar disk almost simultaneously to rule out the alternative hypothesis that the transit-light-source effect could have caused the water feature in TOI-270 b's observed transmission spectrum. Planetary evolution modeling furthermore shows that TOI-270 b could sustain a significant atmosphere on Gyr timescales, despite its high stellar irradiation, if it formed with a large initial volatile inventory.

</div>

<div id="div_fig1">

<img src="tmp_2509.14224/./figures/Three_Scenarios_TOI_270b_titles.png" alt="Fig7" width="100%"/>

**Figure 7. -** Coupled atmosphere-interior evolution modelling shows that a diversity of atmospheric outcomes are consistent with observational constraints. The PACMAN-P geochemical evolution model  (krissansen2024erosion)  was used to explore plausible evolutionary scenarios for TOI-270 b. This model explicitly couples magma ocean solidification, speciation of C, H, O, and Fe-bearing species, stellar evolution, thermal escape, and radiative-convective climate. Subpanels represent scenarios whereby hydrogen escape causes a transition to an $O_2$-$H_2$O atmosphere (top), an $H_2$O-dominated atmosphere (middle), and a case where an $H_2$-dominated atmosphere is retained (bottom). The internal volatile reservoirs of the planet are not shown. (*fig:Three_Scenarios_TOI_270b*)

</div>
<div id="div_fig2">

<img src="tmp_2509.14224/./figures/spec_vs_mods_TOI270b_exoTED_vf.png" alt="Fig3" width="100%"/>

**Figure 3. -** **Top:** Comparison of our measured transmission spectrum, binned at a fixed resolving power of R = 35 for visual clarity, to cloudless self-consistent models produced using SCARLET and assuming a solar carbon-to-oxygen ratio \citep[C/O = 0.55,][]{Asplund2009} and atmosphere metallicities of 1 (yellow), 100 (orange), 300 (maroon), and 1,000 times solar (brown).
The relatively small amplitude of the spectrum of TOI-270 b is inconsistent with any clear $H_2$/He-rich atmosphere with atmosphere metallicities below 300 times solar. Chi-square $\chi^2$ values between the models and the full-resolution ($R=200$, $N=113$ data points)
spectrum are shown for comparison. **Bottom:** Comparison of TOI-270 b with 100 bar pure $H_2$O (blue), $CH_4$(red), $CO_2$(green), and $NH_3$(purple) atmospheres. The rise in transit depth at the shorter wavelengths of NIRSpec/G395H is best reproduced by the absorption features of water. (*fig:spec_vs_mods*)

</div>
<div id="div_fig3">

<img src="tmp_2509.14224/./figures/H2O_vs_pCloud_TOI270b_X_exoTED_vf.png" alt="Fig10.1" width="33%"/><img src="tmp_2509.14224/./figures/H2O_vs_muX_TOI270b_X_exoTED_vf.png" alt="Fig10.2" width="33%"/><img src="tmp_2509.14224/./figures/muX_vs_pCloud_TOI270b_X_exoTED_vf.png" alt="Fig10.3" width="33%"/>

**Figure 10. -** Constraints from the atmospheric retrieval considering the fictitious species X as the background gas. **Left:** Joint constraints on the water mole fraction and effective surface pressure, where color represents the normalized probability density. The 1, 1.5, 2, and 3$\sigma$ probability contours are indicated by the full and dashed black lines for the retrievals assuming no TLS and with constrained TLS, respectively. **Middle:** Joint constraints on the water mole fraction and background gas MMW ($\mu_\mathrm{X}$). The 1.5, 2, and 3$\sigma$ probability contours are indicated by the full and dashed black lines for the retrievals assuming no TLS and with constrained TLS, respectively. Beyond water mole fractions of 90\%, the background MMW is unconstrained as the global MMW is dominated by that of $H_2$O. Below that, the 1.5 and 2$\sigma$ confidence regions are confined to a given range of MMW values, as global MMW that are too high/low would result in water absorption features that are too large/small compared to the tentative feature. **Right:** Joint constraints on the background gas MMW and effective surface pressure. (*fig:h2o_vs_pcloud_muX*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.14224"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

131  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

9  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
